META DATA

    No	Nama Fitur	Penjelasan
    1	floors_before_eq (total)	jumlah total lantai pada bangunan sebelum gempa terjadi
    2	old_building	usia bangunan (tahun)
    3	plinth_area (ft^2)	luas bangunan (ft^2)
    4	height_before_eq (ft)	tinggi bangunan sebelum terjadi gempa (kaki)
    5	land_surface_condition	kondisi permukaan tanah disaat bangunan akan dibangun
    6	type_of_foundation	jenis fondasi yang dipakai untuk bangunan tsb
    7	type_of_roof	jenis atap yang dipakai untuk bangunan tsb
    8	type_of_ground_floor	jenis lantai yang dipakai untuk ground-floor
    9	type_of_other_floor	jenis lantai yang dipakai untuk selain ground-floor
    10	position	Posisi bangunan tersebut apakah bersisian secara langsung dengan bangunan lain
    11	building_plan_configuration	konfigurasi bangunan yang berkaitan dengan bentuk, ukuran, dan penempatan struktur utama bangunan
    12	technical_solution_proposed	Solusi yang ditawarkan untuk bangunan yang terdampak gempa
    13	legal_ownership_status	Status kepemilikan bangunan
    14	has_secondary_use	Keterangan apakah bangunan memiliki kegunaan sekunder
    15	type_of_reinforcement_concrete	Tipe beton bertulang/reinforcement concrete
    16	residential_type	Tipe penggunaaan sebagai hunian
    17	no_family_residing	Jumlah keluarga yang tinggal dalam bangunan tersebut
    18	public_place_type	Tipe penggunaan sebagai tempat umum
    19	industrial_use_type	Tipe industri
    20	govermental_use_type	Tipe penggunaan sebagai bangunan pemerintahan
    21	flexible_superstructure	Keterangan penggunaan superstructure yang fleksibel
    22	wall_binding	Material yang digunakan sebagai perekat bahan pembentuk dinding
    23	wall_material	Material dasar sebagai pembangun dinding
    24	damage_grade	Tingkat kerusakan yang disebabkan oleh gempa (1-5)

    PENJELASAN ISI FITUR 22 dan 23	
    22	wall_binding
    0	Unknown/not stated
    1	Clay
    2	Mortar + Cement
    3	Mortar + Cement, Clay
    5	Mud + Mortar, Clay
    7	Mud + Mortar, Clay, Cement + Mortar
        
    23	wall material
    0	Unknown/not stated
    1	Red Bricks
    2	Stone Bricks
    3	Red Bricks, Stone Bricks

### Import Library

In [2]:
import pandas as pd 
import numpy as np
import regex as re

### Read
read the data train & test

In [4]:
df_train = pd.read_csv('cleanDataset.csv')
df_test = pd.read_csv('cleanTest.csv')

### DATA PREPARATION
dataset terdiri dari 2 jenis data, yaitu float dan object

In [5]:
df_train.isna().sum()

Unnamed: 0                        0
floors_before_eq (total)          0
old_building                      0
plinth_area (ft^2)                0
height_before_eq (ft)             0
land_surface_condition            0
type_of_foundation                0
type_of_roof                      0
type_of_ground_floor              0
type_of_other_floor               0
position                          0
building_plan_configuration       0
technical_solution_proposed       0
legal_ownership_status            0
has_secondary_use                 0
type_of_reinforcement_concrete    0
residential_type                  0
no_family_residing                0
public_place_type                 0
industrial_use_type               0
govermental_use_type              0
flexible_superstructure           0
wall_binding                      0
wall_material                     0
damage_grade                      0
dtype: int64

ada banyak nilai yang null

In [6]:
df_train.select_dtypes(include='float64').columns

Index(['old_building', 'plinth_area (ft^2)', 'height_before_eq (ft)',
       'has_secondary_use', 'no_family_residing', 'damage_grade'],
      dtype='object')

float sebanyak 7

In [7]:
df_train.select_dtypes('object').columns

Index(['land_surface_condition', 'type_of_foundation', 'type_of_roof',
       'type_of_ground_floor', 'type_of_other_floor', 'position',
       'building_plan_configuration', 'technical_solution_proposed',
       'legal_ownership_status', 'type_of_reinforcement_concrete',
       'residential_type', 'public_place_type', 'industrial_use_type',
       'govermental_use_type', 'flexible_superstructure', 'wall_binding',
       'wall_material'],
      dtype='object')

ada 18 columns yg bertipe object

In [8]:
df_train.describe(include='O')

,land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,building_plan_configuration,technical_solution_proposed,legal_ownership_status,type_of_reinforcement_concrete,residential_type,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material
count,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607
unique,3,9,11,10,7,4,10,4,5,4,6,13,9,3,2,6,4
top,Flat,clay sand mixed mortar-stone/brick,bamboo/timber light roof,clay,timber/bamboo-mud,Not attached,Rectangular,Reconstruction,Private,No reinforcement concrete,Non-residential,Non-public,Non-industrial,Non-govermental,unavailable,"Mud + Mortar,Clay",Stone Bricks
freq,250170,124529,79640,202937,169267,239044,289551,182496,289344,284897,287176,301038,279643,301523,277406,242935,244058


#### Feature Floors_Before_EQ

In [9]:
df_train['floors_before_eq (total)'].unique()

array([2, 3, 1, 5, 4, 6, 8, 9, 7], dtype=int64)

story & floor sama aja cmn beda penggunaan katanya


In [10]:
remove_list = ['floor','just','there','is','floor/story','story','has','fl',]
change_list = {'two':2, '1st':1, 'fifth':5, 'three':3, 'one':1 ,'second':2, 'third':3, 'floor-three':3, 'four':4, 'five':5, '3.00':3}
def clear_floors(x):
    if pd.isna(x):
        return x
    else:
        sentences = [y for y in x.lower().split() if y not in remove_list]
        try:
            sentences = int(sentences[0])
        except:
            sentences = change_list[sentences[0]]
        finally:
            return sentences

In [11]:
# df_train['floors_before_eq (total)'].apply(clear_floors)
# df_test['floors_before_eq (total)'].apply(clear_floors)

In [12]:
# df_train['floors_before_eq (total)'] = df_train['floors_before_eq (total)'].apply(clear_floors)
# df_test['floors_before_eq (total)'] = df_test['floors_before_eq (total)'].apply(clear_floors)

#### plinth_area (ft^2)

In [13]:
df_train['plinth_area (ft^2)'].unique()

array([1.706880e+01, 3.002280e+02, 5.638800e+01, 2.743200e+01,
       1.536192e+02, 1.877568e+02, 1.828800e+02, 1.524000e+02,
       3.051048e+02, 1.115568e+02, 4.572000e+01, 9.144000e+01,
       1.066800e+01, 1.091184e+02, 1.524000e+01, 1.097280e+02,
       1.676400e+02, 9.448800e+01, 1.219200e+02, 2.240280e+02,
       1.414272e+02, 1.158240e+02, 1.036320e+01, 1.170432e+02,
       1.310640e+01, 1.143000e+02, 1.493520e+02, 2.377440e+02,
       1.505712e+02, 2.377440e+01, 2.438400e+00, 1.447800e+02,
       1.341120e+02, 2.438400e+01, 2.682240e+01, 1.121664e+02,
       9.448800e+00, 1.371600e+02, 1.167384e+02, 1.371600e+01,
       2.950464e+02, 0.000000e+00, 4.267200e+01, 1.249680e+02,
       1.950720e+01, 5.913120e+01, 9.144000e+00, 1.335024e+02,
       1.234440e+02, 3.962400e+01, 2.956560e+01, 1.676400e+01,
       1.188720e+01, 5.364480e+01, 1.219200e+01, 1.219200e+00,
       2.621280e+01, 1.109472e+02, 3.048000e+00, 1.356360e+02,
       1.508760e+02, 1.036320e+02, 2.011680e+02, 1.2070

In [14]:
def clear_plinth_area(x):
    if pd.isna(x):
        return x
    else:
        sentences = re.sub('[ ft^2]','',x)
        try:
            sentences = int(sentences)
        except:
            sentences = 1001
        return sentences*0.3048

In [15]:
df_train['plinth_area (ft^2)']

0          17.0688
1         300.2280
2          56.3880
3          27.4320
4         153.6192
            ...   
301602     23.1648
301603     13.1064
301604     11.8872
301605     91.4400
301606     91.4400
Name: plinth_area (ft^2), Length: 301607, dtype: float64

In [17]:
# df_train['plinth_area (ft^2)'].apply(clear_plinth_area)
# df_test['plinth_area (ft^2)'].apply(clear_plinth_area)

In [19]:
# df_train['plinth_area (ft^2)'] = df_train['plinth_area (ft^2)'].apply(clear_plinth_area)
# df_test['plinth_area (ft^2)'] = df_test['plinth_area (ft^2)'].apply(clear_plinth_area)

#### land_surface_condition

In [20]:
df_train['land_surface_condition'].unique()

array(['Flat', 'Moderate slope', 'Steep slope'], dtype=object)

In [21]:
df_train['land_surface_condition'].value_counts()

Flat              250170
Moderate slope     41505
Steep slope         9932
Name: land_surface_condition, dtype: int64

Moderate slopes: 10 - 15 % gradient. D. Steep. Steep slopes: 16 - 30 % gradient.

In [22]:
change_list = {'Flat':0,'Moderate slope':1,'Steep slope':2}
def clear_plinth_area(x):
    if pd.isna(x):
        return x
    else:
        return change_list[x]

In [23]:
df_train['land_surface_condition'].apply(clear_plinth_area).value_counts()

0    250170
1     41505
2      9932
Name: land_surface_condition, dtype: int64

In [24]:
df_train['land_surface_condition'] = df_train['land_surface_condition'].apply(clear_plinth_area)
df_test['land_surface_condition'] = df_test['land_surface_condition'].apply(clear_plinth_area)

#### type_of_foundation

In [25]:
df_train['type_of_foundation'].unique()

array(['bamboo or timber', 'clay sand mixed mortar-stone/brick',
       'clay mortar-stone/brick', 'reinforced concrete',
       'cement-stone or cement-brick', 'mud mortar-stone/brick',
       'bamboo/timber', 'others', 'cement-stone/brick'], dtype=object)

In [26]:
df_train['type_of_foundation'].value_counts()

clay sand mixed mortar-stone/brick    124529
clay mortar-stone/brick                99238
mud mortar-stone/brick                 25033
bamboo/timber                          13634
reinforced concrete                    12709
cement-stone or cement-brick           12282
bamboo or timber                        9116
cement-stone/brick                      3230
others                                  1836
Name: type_of_foundation, dtype: int64

In [71]:
type_of_foundation_dict = {
    "mud mortar-stone/brick": "clay mortar-stone/brick",
    "bamboo/timber": "bamboo or timber",
    "bamboo/timber": "bamboo or timber",
    "rc": "reinforced concrete",
    "others": "other",
    "cement-stone or cement-brick": "cement-stone/brick"
}

In [72]:
df_train["type_of_foundation"] = df_train["type_of_foundation"].replace(type_of_foundation_dict).str.lower().str.strip()
df_test["type_of_foundation"] = df_test["type_of_foundation"].replace(type_of_foundation_dict).str.lower().str.strip()

#### type_of_roof

In [68]:
df_train['type_of_roof'].unique()

array(['bamboo or timber light roof',
       'wood light roof or bamboo heavy roof',
       'wood light roof or bamboo light roof', 'rcc/rb/rbc',
       'bamboo/timber heavy roof'], dtype=object)

In [67]:
type_of_roof_dict = {
    "bamboo/timber-heavy roof": "bamboo/timber heavy roof",
    "bamboo/timber-light roof": "bamboo or timber light roof",
    "bamboo/timber light roof": "bamboo or timber light roof ",
    "reinforced cement concrete/rb/rbc": "rcc/rb/rbc",
    "reinforced brick concrete/rcc/rbc": "rcc/rb/rbc",
    "bamboo or timber heavy roof": "bamboo/timber heavy roof",
    "reinforced brick slab/rcc/rbc": "rcc/rb/rbc",
}

df_train["type_of_roof"] = df_train["type_of_roof"].replace(type_of_roof_dict).str.strip()
df_test["type_of_roof"] = df_test["type_of_roof"].replace(type_of_roof_dict).str.strip()


#### type_of_ground_floor

In [75]:
df_train['type_of_ground_floor'].unique()

array(['clay', 'mud', 'reinforced concrete', 'brick or stone',
       'soil, water, loam mixed', 'other', 'lumber', 'timber', 'wood'],
      dtype=object)

In [76]:
type_of_ground_floor_dict = {
    "rc": "reinforced concrete",
    "brick/stone": "brick or stone",
}

df_train["type_of_ground_floor"] = df_train["type_of_ground_floor"].str.lower().replace(type_of_ground_floor_dict)
df_test["type_of_ground_floor"] = df_test["type_of_ground_floor"].str.lower().replace(type_of_ground_floor_dict)


In [34]:
df_train['type_of_ground_floor'].unique()

array(['clay', 'mud', 'reinforced concrete', 'brick or stone',
       'soil, water, loam mixed', 'other', 'lumber', 'timber', 'wood'],
      dtype=object)

#### position

In [35]:
df_train['position'].value_counts()

Not attached       239044
Attached-1 side     51333
Attached-2 side     10725
Attached-3 side       505
Name: position, dtype: int64

#### building_plan_configuration

In [36]:
df_train['building_plan_configuration'].value_counts()

Rectangular                        289551
Square                               6775
L-shape                              4016
T-shape                               400
Multi-projected                       365
Others                                203
U-shape                               174
E-shape                                65
Building with Central Courtyard        33
H-shape                                25
Name: building_plan_configuration, dtype: int64

#### technical_solution_proposed

In [37]:
df_train['technical_solution_proposed'].value_counts()

Reconstruction    182496
Major repair       53633
Minor repair       35843
No need            29635
Name: technical_solution_proposed, dtype: int64

#### legal_ownership_status

In [38]:
df_train['legal_ownership_status'].value_counts()

Private          289344
Public             7686
Institutional      3085
Unknown             902
Other               590
Name: legal_ownership_status, dtype: int64

In [39]:
legal_ownership_status_dict = {
    "Private": "Private",
    "Private Use": "Private",
    "Prvt": "Private",
    "Privste": "Private",
    "Public Use": "Public",
    "Public Space": "Public",
    "Institutional Use": "Institutional",
    "Institutionals": "Institutional",
    "Unknown": "Other",
    "Unspecified": "Other",
}

df_train["legal_ownership_status"] = df_train["legal_ownership_status"].str.strip().replace(legal_ownership_status_dict)
df_test["legal_ownership_status"] = df_test["legal_ownership_status"].str.strip().replace(legal_ownership_status_dict)


In [40]:
df_train['legal_ownership_status'].value_counts()

Private          289344
Public             7686
Institutional      3085
Other              1492
Name: legal_ownership_status, dtype: int64

#### residential_type

In [41]:
df_train["residential_type"] = df_train["residential_type"].replace({
    "Other Residential Type": "Other"
})

df_test["residential_type"] = df_test["residential_type"].replace({
    "Other Residential Type": "Other"
})


#### no_family_residing

In [42]:
df_train["no_family_residing"].value_counts()

1.0     254487
0.0      28362
2.0      15839
3.0       2244
4.0        502
5.0        116
6.0         38
7.0          9
8.0          5
9.0          4
11.0         1
Name: no_family_residing, dtype: int64

In [43]:
df_train["no_family_residing"] = df_train["no_family_residing"].replace({
    "None": 0
})

df_test["no_family_residing"] = df_test["no_family_residing"].replace({
    "None": 0
})

In [44]:
df_train["no_family_residing"].value_counts()

1.0     254487
0.0      28362
2.0      15839
3.0       2244
4.0        502
5.0        116
6.0         38
7.0          9
8.0          5
9.0          4
11.0         1
Name: no_family_residing, dtype: int64

#### public_place_type

In [45]:
df_train["public_place_type"].value_counts()

Non-public                      301038
Shopping Sites                     106
Hospital                            77
Religious Sites                     70
Drugstore                           63
Primary School                      55
Other Institutional Building        42
High School                         41
University                          35
Middle School                       32
Health Clinics                      20
Museum                              18
Recreational Park                   10
Name: public_place_type, dtype: int64

In [46]:
df_train["public_place_type"] = df_train["public_place_type"].apply(lambda x: "Public" if x != "Non-public" else "Non-public")
df_test["public_place_type"] = df_test["public_place_type"].apply(lambda x: "Public" if x != "Non-public" else "Non-public")


In [47]:
df_train["public_place_type"].value_counts()

Non-public    301038
Public           569
Name: public_place_type, dtype: int64

#### industrial_use_type

In [48]:
df_train["industrial_use_type"].value_counts()

Non-industrial     279643
Food                 9831
Service/Tourism      7417
Manufacturing        2141
Metallurgy           1107
Infrastructure       1094
Agro                  167
Forest-based          122
Farm                   85
Name: industrial_use_type, dtype: int64

#### govermental_use_type

In [49]:
df_train["govermental_use_type"].value_counts()

Non-govermental          301523
Police Offices               54
Govermental Buildings        30
Name: govermental_use_type, dtype: int64

#### flexible_superstructure

In [50]:
df_train["flexible_superstructure"].value_counts()

unavailable    277406
available       24201
Name: flexible_superstructure, dtype: int64

#### type_of_other_floor

In [69]:
# 0 for traditional 1 for modern
type_of_other_floor_dict = {
    'timber/bamboo-mud':'timber mud or bamboo-mud',
    'wood or bamboo mud':'wood-mud or bamboo mud',
    'timber-planck':'wood-plank',
    'lumber-plank':'wood-plank',
    'reinforced brick concrete/rcc/rbc':'reinforced cement concrete/rb/rbc',
    'rcc/rb/rbc':'reinforced cement concrete/rb/rbc',
}
df_train.type_of_other_floor = df_train.type_of_other_floor.replace(type_of_other_floor_dict)
df_test.type_of_other_floor = df_test.type_of_other_floor.replace(type_of_other_floor_dict)

### CSV

In [61]:
df_train.type_of_roof.value_counts()

bamboo/timber light roof                79640
wood light roof or bamboo heavy roof    70095
wood light roof or bamboo light roof    59880
bamboo/timber-light roof                40081
bamboo or timber light roof             19974
rcc/rb/rbc                              13868
bamboo/timber heavy roof                 8406
bamboo/timber-heavy roof                 4225
reinforced brick slab/rcc/rbc            2468
bamboo or timber heavy roof              1722
reinforced brick concrete/rcc/rbc        1248
Name: type_of_roof, dtype: int64

In [73]:
df_train.describe(include='O')

,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,building_plan_configuration,technical_solution_proposed,legal_ownership_status,type_of_reinforcement_concrete,residential_type,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material
count,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607,301607
unique,6,5,9,6,4,10,4,4,4,5,2,9,3,2,6,4
top,clay sand mixed mortar-stone/brick,bamboo or timber light roof,clay,timber mud or bamboo-mud,Not attached,Rectangular,Reconstruction,Private,No reinforcement concrete,Non-residential,Non-public,Non-industrial,Non-govermental,unavailable,"Mud + Mortar,Clay",Stone Bricks
freq,124529,139695,202937,169267,239044,289551,182496,289344,284897,287176,301038,279643,301523,277406,242935,244058


In [74]:
df_test.describe(include='O')

,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,building_plan_configuration,technical_solution_proposed,legal_ownership_status,type_of_reinforcement_concrete,residential_type,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material
count,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082,242082
unique,6,5,9,6,4,10,4,4,4,5,2,9,3,2,6,4
top,clay mortar-stone/brick,bamboo or timber light roof,clay,timber mud or bamboo-mud,Not attached,Rectangular,Reconstruction,Private,No reinforcement concrete,Non-residential,Non-public,Non-industrial,Non-govermental,unavailable,"Mud + Mortar,Clay",Stone Bricks
freq,99872,111991,162836,136408,191888,232111,149345,232440,228535,230667,241662,224344,242013,222368,194939,195724


In [55]:
# df_train.drop('Unnamed: 0',axis=1,inplace=True)
# df_train.dropna(inplace=True)

In [56]:
df_train.shape

(301607, 25)

In [77]:
df_train.to_csv('clear_train_fill.csv',sep=',',index=False)
df_test.to_csv('clear_test_fill.csv',sep=',',index=False)

In [58]:
# df_train.corr()['damage_grade']